In [3]:
import pandas as pd 
import numpy as np
import json
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import datetime

In [30]:
df_client = pd.read_csv('./data/predict-energy-behavior-of-prosumers/client.csv')
df_train = pd.read_csv('./data/predict-energy-behavior-of-prosumers/train.csv').sample(100000, random_state=4242)
df_gas_prices = pd.read_csv('./data/predict-energy-behavior-of-prosumers/gas_prices.csv')
df_electricity = pd.read_csv('./data/predict-energy-behavior-of-prosumers/electricity_prices.csv')
df_forecast_weather = pd.read_csv('./data/predict-energy-behavior-of-prosumers/forecast_weather.csv')
df_historical_weather = pd.read_csv('./data/predict-energy-behavior-of-prosumers/historical_weather.csv').sample(100000, random_state=4242)
df_weather_station = pd.read_csv('./data/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv')

In [4]:
df_train.sample(10000)

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
300277,0,0,1,433.396,1,2021-12-10 06:00:00,100,300277,0
1950399,13,0,3,27.155,1,2023-05-10 18:00:00,616,1950399,51
1971589,0,1,2,22.899,1,2023-05-17 07:00:00,623,1971589,61
251822,7,0,2,0.000,0,2021-11-24 05:00:00,84,251822,26
1688322,12,1,3,1.150,0,2023-02-16 23:00:00,533,1688322,49
...,...,...,...,...,...,...,...,...,...
372590,4,1,1,0.000,0,2022-01-03 02:00:00,124,372590,66
862765,0,1,1,228.459,1,2022-06-05 14:00:00,277,862765,4
1955448,15,0,3,203.256,0,2023-05-12 07:00:00,618,1955448,58
940474,15,1,3,356.604,0,2022-06-29 17:00:00,301,940474,60


In [25]:
df_electricity

,forecast_date,euros_per_mwh,origin_date,data_block_id,date,date_str
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1,01-09-21,01-09-21
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1,01-09-21,01-09-21
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1,01-09-21,01-09-21
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1,01-09-21,01-09-21
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1,01-09-21,01-09-21
...,...,...,...,...,...,...
15281,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,637,30-05-23,30-05-23
15282,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,637,30-05-23,30-05-23
15283,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,637,30-05-23,30-05-23
15284,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,637,30-05-23,30-05-23


In [13]:
df_electricity["date"] = pd.to_datetime(df_electricity["forecast_date"])
df_electricity["date"] = df_electricity["date"].apply(lambda x : x.strftime("%d-%m-%y"))
df_electricity["date"]

0        01-09-21
1        01-09-21
2        01-09-21
3        01-09-21
4        01-09-21
           ...   
15281    30-05-23
15282    30-05-23
15283    30-05-23
15284    30-05-23
15285    30-05-23
Name: date, Length: 15286, dtype: object

In [14]:
df_client["date_str"] = df_client["date"].apply(lambda x : str(x))
df_electricity["date_str"] = df_electricity["date"].apply(lambda x : str(x))

Feature Engineering

In [15]:
df_electricity

,forecast_date,euros_per_mwh,origin_date,data_block_id,date,date_str
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1,01-09-21,01-09-21
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1,01-09-21,01-09-21
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1,01-09-21,01-09-21
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1,01-09-21,01-09-21
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1,01-09-21,01-09-21
...,...,...,...,...,...,...
15281,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,637,30-05-23,30-05-23
15282,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,637,30-05-23,30-05-23
15283,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,637,30-05-23,30-05-23
15284,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,637,30-05-23,30-05-23


Creation Colonne 

In [16]:
# crétaion de la colonne "temperature" avec une valeur initiale de -1
df_train["price_elec"] = -1

liste_date = df_train["datetime"].unique()

for i in range(len(liste_date)):
    if(i % 50 ==0):
        print(i, "/", len(liste_date), end="\r")
unique_date =liste_date[i]
for unique_date in df_train["datetime"].unique():
  # Récuperer le prix de l'electricite a ce jour la
  df_elec_filtered = df_electricity[df_electricity["origin_date"] == unique_date]
  if len(df_elec_filtered) != 0:
    if len(df_elec_filtered) == 1:
      elec_price = df_elec_filtered["euros_per_mwh"]
    else : 
      elec_price = df_elec_filtered.iloc[0]["euros_per_mwh"]

    # Pour toutes les lignes de df_train a ce jour la, insérer le prix
    df_train_filtered = df_train[df_train["datetime"] == unique_date]
    df_train.loc[df_train_filtered.index, "price_elec"] = elec_price.values[0]

C:\Users\Jawad\AppData\Local\Temp\ipykernel_14556\3953736527.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '199.99' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_train.loc[df_train_filtered.index, "price_elec"] = elec_price.values[0]


In [20]:
df_train

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,price_elec
1414989,11,0,3,790.836,1,2022-11-24 01:00:00,449,1414989,45,199.99
953684,9,1,3,4.396,0,2022-07-03 21:00:00,305,953684,37,211.31
103626,4,0,3,0.000,0,2021-10-06 05:00:00,35,103626,16,98.25
1745132,15,1,1,0.000,0,2023-03-06 19:00:00,551,1745132,59,129.23
1961232,7,1,1,0.000,0,2023-05-14 02:00:00,620,1961232,29,85.59
...,...,...,...,...,...,...,...,...,...,...
930074,5,1,0,1004.700,0,2022-06-26 12:00:00,298,930074,21,268.00
955623,4,0,3,1.938,1,2022-07-04 12:00:00,306,955623,16,150.00
1042698,15,1,0,0.000,0,2022-07-31 23:00:00,333,1042698,64,471.48
1928077,4,1,0,734.972,1,2023-05-03 22:00:00,609,1928077,17,101.89


Ajout de température 

In [31]:
df_historical_weather

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
43867,2021-09-17 07:00:00,7.3,4.3,0.0,0.00,1014.8,10,11,0,0,4.277778,79,28.0,13.0,15.0,59.1,24.2,17.0
1353413,2023-01-17 12:00:00,8.1,4.8,0.0,0.00,1005.8,85,94,0,0,9.916667,261,0.0,0.0,0.0,57.6,23.2,505.0
256936,2021-12-05 14:00:00,-12.3,-15.6,0.0,0.00,1007.3,88,98,0,0,4.055556,40,100.0,35.0,65.0,57.6,25.7,97.0
1358726,2023-01-19 11:00:00,0.8,-0.2,0.0,0.35,990.8,100,90,99,87,9.083333,259,28.0,0.0,28.0,58.5,26.7,507.0
1223014,2022-11-29 23:00:00,3.2,-0.8,0.1,0.00,1033.6,93,87,25,0,7.472222,305,0.0,0.0,0.0,59.4,21.7,456.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530421,2023-03-24 08:00:00,4.6,4.0,0.0,0.00,992.9,92,100,1,3,5.555556,247,18.0,0.0,18.0,58.5,26.2,570.0
1030395,2022-09-19 07:00:00,10.0,8.3,0.0,0.00,996.9,100,80,71,75,1.833333,99,33.0,14.0,19.0,59.7,26.2,384.0
1650562,2023-05-08 01:00:00,4.0,0.1,0.0,0.00,1032.4,89,0,99,100,2.000000,198,0.0,0.0,0.0,57.9,22.7,615.0
1251051,2022-12-10 10:00:00,-10.8,-11.8,0.0,0.00,1004.7,90,100,0,0,3.222222,306,20.0,2.0,18.0,57.6,26.2,466.0


In [32]:
df_historical_weather["datetime"].nunique()

df_historical_weather["date"] = pd.to_datetime(df_historical_weather["datetime"])
df_historical_weather["date"] = df_historical_weather["date"].apply(lambda x : x.strftime("%d-%m-%y"))
df_historical_weather["date"]

43867      17-09-21
1353413    17-01-23
256936     05-12-21
1358726    19-01-23
1223014    29-11-22
             ...   
1530421    24-03-23
1030395    19-09-22
1650562    08-05-23
1251051    10-12-22
901090     02-08-22
Name: date, Length: 100000, dtype: object

In [33]:
df_historical_weather["date_str"] = df_historical_weather["date"].apply(lambda x : str(x))

In [34]:
df_historical_weather

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id,date,date_str
43867,2021-09-17 07:00:00,7.3,4.3,0.0,0.00,1014.8,10,11,0,0,4.277778,79,28.0,13.0,15.0,59.1,24.2,17.0,17-09-21,17-09-21
1353413,2023-01-17 12:00:00,8.1,4.8,0.0,0.00,1005.8,85,94,0,0,9.916667,261,0.0,0.0,0.0,57.6,23.2,505.0,17-01-23,17-01-23
256936,2021-12-05 14:00:00,-12.3,-15.6,0.0,0.00,1007.3,88,98,0,0,4.055556,40,100.0,35.0,65.0,57.6,25.7,97.0,05-12-21,05-12-21
1358726,2023-01-19 11:00:00,0.8,-0.2,0.0,0.35,990.8,100,90,99,87,9.083333,259,28.0,0.0,28.0,58.5,26.7,507.0,19-01-23,19-01-23
1223014,2022-11-29 23:00:00,3.2,-0.8,0.1,0.00,1033.6,93,87,25,0,7.472222,305,0.0,0.0,0.0,59.4,21.7,456.0,29-11-22,29-11-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530421,2023-03-24 08:00:00,4.6,4.0,0.0,0.00,992.9,92,100,1,3,5.555556,247,18.0,0.0,18.0,58.5,26.2,570.0,24-03-23,24-03-23
1030395,2022-09-19 07:00:00,10.0,8.3,0.0,0.00,996.9,100,80,71,75,1.833333,99,33.0,14.0,19.0,59.7,26.2,384.0,19-09-22,19-09-22
1650562,2023-05-08 01:00:00,4.0,0.1,0.0,0.00,1032.4,89,0,99,100,2.000000,198,0.0,0.0,0.0,57.9,22.7,615.0,08-05-23,08-05-23
1251051,2022-12-10 10:00:00,-10.8,-11.8,0.0,0.00,1004.7,90,100,0,0,3.222222,306,20.0,2.0,18.0,57.6,26.2,466.0,10-12-22,10-12-22


In [38]:
# Ajouter la colonne "temperature" avec une valeur initiale de -1
df_train["temperature"] = -1

# Parcourir les dates uniques dans df_train
for unique_date in df_train["datetime"].unique():
    # Filtrer le DataFrame df_historical_weather pour la date unique actuelle
    df_weather_filtered = df_historical_weather[df_historical_weather["datetime"] == unique_date]
    
    # Vérifier s'il y a des données pour cette date
    if len(df_weather_filtered) != 0:
        # Extraire la température pour cette date
        temperature_value = df_weather_filtered.iloc[0]["temperature"]
        
        # Filtrer le DataFrame df_train pour la date unique actuelle
        df_train_filtered = df_train[df_train["datetime"] == unique_date]
        
        # Mettre à jour la colonne "temperature" pour toutes les lignes correspondant à cette date dans df_train
        df_train.loc[df_train_filtered.index, "temperature"] = temperature_value


C:\Users\Jawad\AppData\Local\Temp\ipykernel_14556\4121195767.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-4.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_train.loc[df_train_filtered.index, "temperature"] = temperature_value


In [37]:
df_train

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,temperature
1414989,11,0,3,790.836,1,2022-11-24 01:00:00,449,1414989,45,-4.8
953684,9,1,3,4.396,0,2022-07-03 21:00:00,305,953684,37,19.0
103626,4,0,3,0.000,0,2021-10-06 05:00:00,35,103626,16,8.7
1745132,15,1,1,0.000,0,2023-03-06 19:00:00,551,1745132,59,-4.0
1961232,7,1,1,0.000,0,2023-05-14 02:00:00,620,1961232,29,9.4
...,...,...,...,...,...,...,...,...,...,...
930074,5,1,0,1004.700,0,2022-06-26 12:00:00,298,930074,21,28.5
955623,4,0,3,1.938,1,2022-07-04 12:00:00,306,955623,16,23.8
1042698,15,1,0,0.000,0,2022-07-31 23:00:00,333,1042698,64,18.0
1928077,4,1,0,734.972,1,2023-05-03 22:00:00,609,1928077,17,4.0
